In [3]:
import torch
import gc

torch.cuda.empty_cache()  # Clears cached memory
gc.collect()

0

In [4]:



#mixed precision
#comments for CNN 
#readme
#chess
#varying the parameters of the model to optimize the performance

import numpy as np
print(np.__version__)


import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

from tqdm.notebook import trange

import random
import math
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm import trange
import torch
import gc


2.1.2
2.5.1+cu121


In [15]:
import numpy as np

class ConnectFour:
    def __init__(self):
        self.row_count = 6
        self.column_count = 7
        self.action_size = self.column_count
        self.in_a_row = 4
        
    def __repr__(self):
        return "ConnectFour"
        
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    def get_next_state(self, state, action, player):
        row = np.max(np.where(state[:, action] == 0))
        state[row, action] = player
        return state
    
    def get_valid_moves(self, state):
        return (state[0] == 0).astype(np.uint8)
    
    def check_win(self, state, action): #state is the current state and action is the action taken by the opponent
        if action == None:
            return False
        
        row = np.min(np.where(state[:, action] != 0))
        column = action
        player = state[row][column]

        def count(offset_row, offset_column):
            for i in range(1, self.in_a_row):
                r = row + offset_row * i
                c = action + offset_column * i
                if (
                    r < 0 
                    or r >= self.row_count
                    or c < 0 
                    or c >= self.column_count
                    or state[r][c] != player
                ):
                    return i - 1
            return self.in_a_row - 1

        return (
            count(1, 0) >= self.in_a_row - 1 # vertical
            or (count(0, 1) + count(0, -1)) >= self.in_a_row - 1 # horizontal
            or (count(1, 1) + count(-1, -1)) >= self.in_a_row - 1 # top left diagonal
            or (count(1, -1) + count(-1, 1)) >= self.in_a_row - 1 # top right diagonal
        )
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False
    
    def get_opponent(self, player):
        return -1*player
    
    def get_opponent_value(self, value):
        return -1*value
    
    def change_perspective(self, state, player):
        return state * player
    
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        if len(state.shape) == 3:
            encoded_state = np.swapaxes(encoded_state, 0, 1)
        
        return encoded_state

    def computer_play(self):
        return  random.randint(0,self.column_count-1)
        



        
        


In [16]:
class ResNet(nn.Module):

    def __init__(self,game,num_resBlocks,num_hidden, in_channels):
        super().__init__()
        #input dimension is : 100x3x6x7 
        #kernel = convolutional matrix 
        #num_hidden = number of filters
        #kernel_size 3x3
        self.num_resBlocks = num_resBlocks
        self.in_channels = in_channels #this is the channel size of the matrice, the reason this is  3 because the cases where player 1 plays , where player 2 plays and nobody plays is represented by 3 different layers where each is represented by 1 and the rest being 0 for connect 4. 
        self.out_channels = num_hidden # channel size of the output not the width and length  
        self.startBlock = nn.Sequential(
            nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1), #3 6 7 -> num_hidden 6 7
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU()
        )
        self.BackBone = nn.ModuleList([ResBlock(self.out_channels) for i in range(self.num_resBlocks)] ) #We use modulelist instead of sequential because it allows us to use skip connections 
        self.value_head = nn.Sequential(
            nn.Conv2d(self.out_channels, 3, kernel_size=3, padding=1), #DONT KNOW WHY 32 yet may be empirical
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3*game.row_count*game.column_count, 1),
            nn.Tanh()  #value is between -1 and 1
        )
        self.policy_head = nn.Sequential(
            nn.Conv2d(self.out_channels, 32, kernel_size=3, padding=1), #DONT KNOW WHY 32 yet 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*game.row_count*game.column_count, game.action_size), #the action size for each game may be different 
            
        )
        self.to(device)
   

    def forward(self,state): 
        
        x = self.startBlock(state)
        for block in self.BackBone:
            x = block(x)  # Apply each ResBlock manually
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value

        

class ResBlock(nn.Module):
    def __init__(self,out_channels):
        super().__init__()
        self.out_channels = out_channels
        self.Conv2d_1 = nn.Conv2d(self.out_channels,self.out_channels,kernel_size=3,padding=1)
        self.BatchNorm_1 = nn.BatchNorm2d(self.out_channels)
        self.Conv2d_2 = nn.Conv2d(self.out_channels,self.out_channels,kernel_size=3,padding=1)
        self.BatchNorm_2 = nn.BatchNorm2d(self.out_channels)

    def forward(self,x):
        temp = x
        x = self.BatchNorm_2(self.Conv2d_2(F.relu(self.BatchNorm_1(self.Conv2d_1(x)))))
        x = temp + x
        x = F.relu(x)
        return x 

        


In [17]:
class Node:
     def __init__(self,state,args,transition_action, player,game):
          self.state = state 
          self.nb_visits = 0
          self.value  = 0 
          self.prior_proba = 0
          self.children = []
          self.parent  = None 
          self.args = args      
          self.transition_action = transition_action
          self.player = player 
          self.game = game


     def get_prior_proba(self):
          return self.prior_proba
     def get_player(self):
          return self.player
     def get_transition_action(self):
          return self.transition_action

     def get_visits(self):
          return self.nb_visits
     def get_state(self):
          return self.state
     def set_value(self, value ):
          self.value = value

     def get_value(self):
          return self.value
     def set_prior_proba(self,proba):
          self.prior_proba = proba

     def get_children(self):
          return self.children

     def add_child(self, child_node,transition_action):
          self.children.append((child_node,transition_action))
          child_node.set_parent(self)
          
     def set_parent(self, parent_node):
          self.parent = parent_node

     def get_parent(self):
          return self.parent 

     def calc_PUCT(self):
          parent = self.get_parent() 
          parent_visits = parent.get_visits()
          if self.nb_visits == 0:
            q_value = 0
          else:
               q_value = 1 - ((self.value / self.nb_visits) + 1) / 2
          res = q_value + self.args['C'] * (math.sqrt(parent_visits) / (self.nb_visits + 1)) *  self.prior_proba
          #print(f"res with nb_visits = ",self.nb_visits, " and parent_visits = ", parent_visits, " is ", res)
          return res

     def choose_best_child(self):
          print("--------")
          for child in self.children:
               print("child.nb_visits = ", child[0].get_visits())
          if self.children != []:
               best_tuple = max(self.children, key=lambda item: item[0].calc_PUCT()) #we choose the best child node based on the highest PUCT value
               best_child = best_tuple[0]
               best_action = best_tuple[1]

               return best_child, best_action
          else :
              return None, None


     def increment_visit(self):
          self.nb_visits += 1 

     def add_value(self, value):
          self.value += value 
     def selfPlay_policy(self):
          action_size = 7
          total_visits = sum(child[0].get_visits() for child in self.children) + 1e-8
          policy = [0] * action_size

          for child_node, _ in self.children:
               action = child_node.get_transition_action()
               if action is not None:
                    # Use += in case there are multiple child nodes for the same action (edge case)
                    policy[action] += child_node.get_visits() / total_visits

          return policy

     def is_fully_expanded(self):
          if self.children == []:
               return False
          else:
               return True
                    
     def expand(self,probas):
          for i, proba in enumerate(probas) :  
                #either we get all the actions or the all allowed actions I am not sure but the dimensions of the policy returned by the ResNet must equal the number of actions / states 
                if proba > 0 : 
                    child_state = self.state.copy()
                    child_state = self.game.get_next_state(child_state, i, 1)
                    child_state = self.game.change_perspective(child_state, player=-1)#dont forget to change the perspective every time a new node is added
                    print("child_state in expand   = ", child_state)
                    child_node = Node(child_state,self.args,i,self.player*-1,self.game)
                    child_node.set_prior_proba(proba)
                    child_node.set_parent(self)
                    self.add_child(child_node,i)


     def backpropagate(self, value):
        self.value += value
        self.nb_visits += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  

In [19]:
"""class MCTS:
    def __init__(self,game,args,model,states):
        
        self.game =game 
        self.root_nodes = [Node(state,args,None,1) for state in states ]
        self.args = args
        self.model=model 


    def parallel_selection(self):
        # In MCTS.parallel_selection
        sample_state = self.root_nodes[0].get_state() if self.root_nodes else None
        nodes_and_actions = []
        nodes_and_actions = [(self.selection(node)) for node in self.root_nodes] #we select the best child for each root node

        return nodes_and_actions
    
    def selection(self,root):
        current_node = root 
        if current_node.get_children() == []:
            return current_node, None

        while True:
                
                next_node, action = current_node.choose_best_child()
                
                if next_node is None:
                    return current_node, action  
                
                current_node = next_node
                
                

    #@torch.no_grad() # EXTREMELY IMPORTANT IF NOT THE GPU MEMORY FILLS UP 
    def expansion(self,nodes_and_actions):
        if not nodes_and_actions:
            return
        
        remaining_nodes = []        
        for node, action in nodes_and_actions:
           value, is_terminal = self.game.get_value_and_terminated(node.get_state(), action)
           if is_terminal:
               self.backprop(node, value) #we only backpropagate the terminal nodes in the less computationally expensive version of the MCTS
           else:
               remaining_nodes.append((node, action))
        if not remaining_nodes:
           return
        
        states_encoded = np.stack([self.game.get_encoded_state(np.array(node.get_state())) for node, action in remaining_nodes]) #np.stack stacks the arrays along the first axis meaning it creates a single contigous array instead of an array of arrays
        states_not_encoded = [np.array(node.get_state()) for node, action in remaining_nodes]
        states_tensor = torch.tensor(states_encoded, device=device, dtype=torch.float32)
        with torch.no_grad(): 
            policies, values = self.model.forward(states_tensor)
        policies = torch.softmax(policies, axis=1).detach().cpu().numpy()
        policies = (1 - self.args['dirichlet_epsilon']) * policies + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)
        
        valid_moves = np.array([self.game.get_valid_moves(state) for state in states_not_encoded])
        policies = policies * valid_moves # Normalize each policy row
        policies /= np.sum(policies)
        #the dirichlet noise is added to the policy to encourage exploration
        """

"""for i, node_and_action in enumerate(remaining_nodes):
            node, action = node_and_action
            node.set_value(values[i])
            self.backprop(node, values[i]) #we backpropagate the value of the node to the parent nodes""" #backprop is done for every evaluation in the original alphazero paper but is computationally expensive removing this saved A LOT of time
       """ for j, policy in enumerate(policies):
            state = states_not_encoded[j]
            node, action = remaining_nodes[j]
            modified_policy = policies[j]
            value = values[j]
            self.backprop(node, value) #we backpropagate the value of the node to the parent nodes
            for i, proba in enumerate(modified_policy) :  
                #either we get all the actions or the all allowed actions I am not sure but the dimensions of the policy returned by the ResNet must equal the number of actions / states 
                if proba > 0 : 
                    player = self.game.get_opponent(node.get_player())
                    next_state = self.game.get_next_state(state, i, player)
                    next_state = self.game.change_perspective(next_state, player) #dont forget to change the perspective every time a new node is added
                    child_node = Node(next_state,self.args,i,player)
                    child_node.set_prior_proba(proba)
                    child_node.set_parent(node)
                    child_node.nb_visits = 1
                    node.add_child(child_node,i)"""
                    

                    

        

   """ def backprop(self, node, value):
        node.value += value 
        node.nb_visits += 1
        parent = node.get_parent()
        if parent != None:
            value = self.game.get_opponent_value(value)
            self.backprop(parent, value) #using recursion instead of while loop is more efficient
        else:
            return
        

    def perform_episode(self):

        nodes_and_actions = self.parallel_selection()
        self.expansion(nodes_and_actions) #we perform an episode by performing selection, expansion and backpropagation
        
    def get_possible_actions(self,node):
        possible_acitons = self.game.get_valid_moves(node.get_state())
        return possible_acitons
    
    def round_of_selfPlay(self):
        #we perform MCTS_iterations number of episodes and return the number of visits for each action which is forms the policy for the state
        total_nodes = sum(len(node.get_children()) for node in self.root_nodes)
        for iteration in range(self.args["MCTS_iterations"]):
            self.perform_episode()


        true_probas = [node.selfPlay_policy() for node in self.root_nodes]

        return true_probas"""
        



IndentationError: unindent does not match any outer indentation level (<string>, line 91)

In [20]:
class MCTSParallel:

    def __init__(self,game,args,model):
        self.game =game 
        self.args = args
        self.model=model 


    @torch.no_grad() # EXTREMELY IMPORTANT IF NOT THE GPU MEMORY FILLS UP
    def search(self,states,spGames,player):
            

        policies, values =  self.model.forward(torch.tensor(self.game.get_encoded_state(states), dtype=torch.float32, device=device))
        policies = torch.softmax(policies, axis=1).cpu().numpy()
        policies = (1 - self.args['dirichlet_epsilon']) * policies + self.args['dirichlet_epsilon'] \
        * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size, size=policies.shape[0])
        #for the current root state we are in in every game get its policy
        valid_moves = np.array([self.game.get_valid_moves(state) for state in states])
        #print("len(states) = ", len(states))
        #print("len(spGames) = ", len(spGames))
        for i in range(len(states)):
            spGames[i].root = Node(states[i],self.args,i,player,self.game) #we create a new root node for each game and set its visit count to 1
            policy = policies[i] * valid_moves[i] #we multiply the policy by the valid moves to get the valid policy
            policy /= np.sum(policy) #we normalize thestates,spGames policy to get the probabilities of each action
            spGames[i].root.expand(policy) # we only add the dirichlet noise at the root node


        for _ in range(self.args["MCTS_iterations"]):
            for i in range(len(spGames)):
                spGames[i].node = None
                leaf_node = spGames[i].root
                while leaf_node.is_fully_expanded():
                    leaf_node, action_taken_by_opponent = leaf_node.choose_best_child() 
                #we go down the tree until we reach a leaf of the root node of the spgame
                value, is_terminal = self.game.get_value_and_terminated(leaf_node.get_state(), leaf_node.transition_action)
                value = self.game.get_opponent_value(value) 
                if is_terminal:
                    leaf_node.set_value(value)
                    leaf_node.backpropagate(value)
                    spGames[i].node = None
                else: 
                    spGames[i].node = leaf_node
                #if we achieved a terminal state we backpropagate and this spgame is done if not we set spg.node to the leaf node to indicate that the game isnt over

            not_finished_games =  [spGame for spGame in spGames if spGame.node != None]
            #we get the not over games
            if not_finished_games  == []:
                continue 
            rollout_states = torch.tensor(
                np.array([self.game.get_encoded_state(spGame.node.get_state()) for spGame in not_finished_games]),
                dtype=torch.float32,
                device=device  
            )

            policies, values = self.model.forward(rollout_states)
            policies = torch.softmax(policies, axis=1).cpu().numpy()
            values = values.cpu().numpy()
            for i in range(len(not_finished_games)):
                valid_moves  = self.game.get_valid_moves(not_finished_games[i].node.get_state())
                true_policy = policies[i] * valid_moves
                true_policy /= np.sum(true_policy) 
                value = values[i]
                not_finished_games[i].node.set_value(value)
                not_finished_games[i].node.backpropagate(value)
                not_finished_games[i].node.expand(true_policy)
            

         
         
         
         
        



    

In [21]:
def evaluate_vs_random(model, game, num_games=10):
    
    wins, draws, losses = 0, 0, 0

    for _ in range(num_games):
        state = game.get_initial_state()
        player = 1
        while True:
            if player == 1:
                # Model's turn
                encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)
                with torch.no_grad():
                    policy_logits, _ = model(encoded)
                    policy = torch.softmax(policy_logits, dim=1).cpu().numpy()[0]
                    valid_moves = game.get_valid_moves(state)
                    policy *= valid_moves
                    if policy.sum() == 0:
                        policy = valid_moves / valid_moves.sum()
                    else:
                        policy /= policy.sum()
                    action = np.random.choice(game.action_size, p=policy)

            else:
                # Random agent's turn
                valid_moves = game.get_valid_moves(state)
                action = np.random.choice(np.flatnonzero(valid_moves))

            state = game.get_next_state(state, action, player)
            value, is_terminal = game.get_value_and_terminated(state, action)
            if is_terminal:
                if value == 1:
                    if player == 1:
                        wins += 1
                    else:
                        losses += 1
                else:
                    draws += 1
                break
            player = game.get_opponent(player)

    total = wins + draws + losses
    print(f"Model vs Random — Wins: {wins}, Draws: {draws}, Losses: {losses} ({total} games)")


In [22]:

class AlphaZero():



    def __init__(self,game,args,model,optimizer):
        self.game = game
        self.model = model
        self.args = args 
        self.replay_buffer = [] #we store the historical data for the neural network valeus to be corrected here 
        self.current_game_tree =  SPG(game,args)
        self.optimizer = optimizer
        self.MCTS = MCTSParallel(self.game,self.args,self.model)

    def game_of_SelfPlay(self,spGames): 
            
            player  = 1
            action = None
            
            self.model.eval()
            to_remove = []
            while len(spGames) > 0: 
                states = np.stack([spg.state for spg in spGames])
                neutral_states = self.game.change_perspective(states, player)
                self.MCTS.search(neutral_states,spGames,player)
                
                
                for i in range(len(spGames))[::-1]:
                    spg = spGames[i]
                      
                    single_policy = np.zeros(self.game.action_size)
                    for child,action in spg.root.get_children():
                        single_policy[action] = child.nb_visits
                    single_policy /= np.sum(single_policy)

                    #print(f"we are appending state to memory for game {i}: \n",state)
                    #print(f"spGames[{i}].memory:\n ",spGames[i].memory)
                    #print("state, true_policy, player: ",state, true_policy, player)
                    
                    

                    true_policy = single_policy
                    spGames[i].memory.append((spg.root.state,true_policy,player))

                    #With high temperature: probabilities become more uniform
                    #With low temperature: differences get exaggerated, making the highest probability action even more likely
                    action = np.random.choice(len(single_policy), p=single_policy)
                    spg.state = self.game.get_next_state(spg.state, action, player)
                    
                    game_value, is_terminal = self.game.get_value_and_terminated(spg.state, action)
                    #once an spg is finished we store the data that we stored in spg.memory in the replay buffer with the value of the end game and we remove the spg from the list
                    #no need to keep track of the node tree since every state is already stored inside the spg memory 
                    if is_terminal:
                         #seems like we are either putting 1 or 0 as the value of the position instead of a non discrete value
                         # Over time, the value target for a position becomes an average of many discrete outcomes, which can be a non-discrete float

                        
                        for hist_state,true_policy,hist_player in spg.memory:

                            hist_outcome = game_value if hist_player == player else self.game.get_opponent_value(game_value)
                            print("\n[Replay Buffer Debug]")
                            print(f"Final game_value: {game_value}")
                            print(f"Final player to move: {player}")
                            print(f"Hist player: {hist_player}")
                            print(f"Flipped value stored: {hist_outcome}")
                            print(f"State:\n{hist_state}")
                            print(f"True policy:\n{true_policy}")

                            self.replay_buffer.append((
                            self.game.get_encoded_state(hist_state),
                            true_policy,
                            hist_outcome))
                        del spGames[i]
                       
                player = self.game.get_opponent(player)



    def train(self):
            random.shuffle(self.replay_buffer)
            for j in trange(0, len(self.replay_buffer),self.args["batch_size"], desc="Training", leave=False):
                batch = self.replay_buffer[j:min(j+self.args["batch_size"], len(self.replay_buffer))]
                states, true_policies, values = zip(*batch)
                states, true_policies, values = np.array(states), np.array(true_policies), np.array(values)
                states = torch.tensor(states, dtype=torch.float32, device=device)
                policy_targets = torch.tensor(true_policies, dtype=torch.float32, device=device)
                value_targets = torch.tensor(values, dtype=torch.float32, device=device).view(-1, 1)
                policy_logits, value_logits = self.model.forward(states)
                print("\n=== Training Sample ===")
                
                print(f"Target Value: {value_targets[0].item():.4f}")
                print(f"State:\n{states[0].cpu()}")
                print(f"Policy Target:\n{policy_targets[0].cpu()}")
                print(f"Policy Logits:\n{policy_logits[0].cpu()}")
                policy_loss = F.cross_entropy(policy_logits,policy_targets)
                #F.kl_div is better for soft class loss calculations 
                """policy_loss = F.kl_div(
                    F.log_softmax(policy_logits, dim=1),
                    policy_targets,
                    reduction='batchmean'
                )"""

                value_loss = F.mse_loss(value_logits,value_targets)
                loss = policy_loss + value_loss
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                print(f"value_loss : {value_loss}, policy_loss : {policy_loss}, loss : {loss}")


    def learn(self):   
        if self.args["load_model"] == True:
            self.model.load_state_dict(torch.load(f"model_{self.game}.pt"))
            self.optimizer.load_state_dict(torch.load(f"optimizer_{self.game}.pt"))   
        for  i in range(self.args['iterations']):
            for i in trange(self.args['num_parallel_selfplays']):
                spGames = [SPG(self.game,self.args) for h in range(self.args["parallel_games"])]     
                self.game_of_SelfPlay(spGames)
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train()
            self.replay_buffer = []
            if i % self.args["save_frequency"] == 0:
                torch.save(self.model.state_dict(), f"model_{self.game}.pt")
                torch.save(self.optimizer.state_dict(), f"optimizer_{self.game}.pt")

            
        

    def plot_metrics(self):
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(12, 8))
        
        # Plot losses
        plt.subplot(2, 1, 1)
        plt.plot(self.iterations, self.value_losses, label='Value Loss')
        plt.plot(self.iterations, self.policy_losses, label='Policy Loss')
        plt.title('AlphaZero Training Losses')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        
        # Plot total loss
        plt.subplot(2, 1, 2)
        plt.plot(self.iterations, self.total_losses, label='Total Loss', color='green')
        plt.title('AlphaZero Total Loss')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig(f'alphazero_metrics_{len(self.iterations)}.png')
        plt.close()
        
    def free_memory(self):
        torch.cuda.empty_cache()
        gc.collect()
class SPG():
    
    def __init__(self,game,args):
        self.game = game
        self.memory = []
        self.state = game.get_initial_state()
        self.root = None
        self.node = None
        


         



In [23]:
#Testing the ResNet model

game = ConnectFour()
model = ResNet(game, 9, 128, 3).to(device)

# Generate a dummy board
state = game.get_initial_state()
encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)

# Run a forward pass
policy_logits, value = model(encoded)
policy_probs = torch.softmax(policy_logits, dim=1)

print("Policy shape:", policy_probs.shape)
print("Value prediction:", value.item())
print("Policy probs (sum):", policy_probs.sum().item())
print("Top move prob:", policy_probs.max().item())


Policy shape: torch.Size([1, 7])
Value prediction: 0.31358757615089417
Policy probs (sum): 0.9999999403953552
Top move prob: 0.2724999785423279


In [ ]:
#Testing a round of SelfPlay
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

args = {
    'C': 2,
    'MCTS_iterations': 20,
    'num_parallel_selfplays': 5,
    'iterations': 5,
    'parallel_games': 30,
    'batch_size': 64,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3,
    'temperature': 1.25,
    'load_model': False,
    'save_frequency': 5,
    'num_epochs': 4,
}
alphaZero = AlphaZero(game,args,model,optimizer)
alphaZero.learn()

In [13]:

evaluate_vs_random(model, game, num_games=10)

Model vs Random — Wins: 5, Draws: 0, Losses: 5 (10 games)


In [107]:
#model.load_state_dict(torch.load("model_ConnectFour.pt"))

import pygame
import numpy as np
import torch
import time

# Game settings
ROWS, COLS = 6, 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
WIDTH, HEIGHT = COLS * SQUARESIZE, (ROWS + 1) * SQUARESIZE

# Colors
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

def draw_board(screen, board):
    screen.fill(BLACK)
    for c in range(COLS):
        for r in range(ROWS):
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, (r+1)*SQUARESIZE, SQUARESIZE, SQUARESIZE))
            piece = board[r][c]
            color = BLACK
            if piece == 1:
                color = RED
            elif piece == -1:
                color = YELLOW
            pygame.draw.circle(screen, color, (int(c*SQUARESIZE + SQUARESIZE/2), int((r+1)*SQUARESIZE + SQUARESIZE/2)), RADIUS)
    pygame.display.update()

def get_column_from_mouse(x):
    return int(x / SQUARESIZE)

def get_row_to_drop(board, col):
    for r in range(ROWS-1, -1, -1):
        if board[r][col] == 0:
            return r
    return None

def play_against_model_pygame(model, game, device, play_as=1):
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    font = pygame.font.SysFont("monospace", 50)
    pygame.display.set_caption("Connect Four - Play vs Model")

    board = np.zeros((ROWS, COLS), dtype=int)
    state = game.get_initial_state()
    player = 1
    game_over = False

    draw_board(screen, board)

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            if player == play_as and event.type == pygame.MOUSEBUTTONDOWN:
                col = get_column_from_mouse(event.pos[0])
                valid_moves = game.get_valid_moves(state)
                if valid_moves[col] == 1:
                    row = get_row_to_drop(board, col)
                    board[row][col] = player
                    state = game.get_next_state(state, col, player)
                    value, is_terminal = game.get_value_and_terminated(state, col)
                    draw_board(screen, board)

                    if is_terminal:
                        game_over = True
                        text = "You win!" if value == 1 else "Draw!" if value == 0 else "Model wins!"
                        label = font.render(text, True, RED if value == 1 else YELLOW)
                        screen.blit(label, (40, 10))
                        pygame.display.update()
                        time.sleep(3)
                        break

                    player = game.get_opponent(player)

        if player != play_as and not game_over:
            pygame.time.wait(500)
            encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)
            with torch.no_grad():
                policy_logits, _ = model(encoded)
                policy = torch.softmax(policy_logits, dim=1).cpu().numpy()[0]
                valid_moves = game.get_valid_moves(state)
                policy *= valid_moves
                if policy.sum() == 0:
                    policy = valid_moves / valid_moves.sum()
                else:
                    policy /= policy.sum()
                action = np.random.choice(game.action_size, p=policy)

            row = get_row_to_drop(board, action)
            board[row][action] = player
            state = game.get_next_state(state, action, player)
            value, is_terminal = game.get_value_and_terminated(state, action)
            draw_board(screen, board)

            if is_terminal:
                game_over = True
                text = "Model wins!" if value == 1 else "Draw!" if value == 0 else "You win!"
                label = font.render(text, True, YELLOW if value == 1 else RED)
                screen.blit(label, (40, 10))
                pygame.display.update()
                time.sleep(3)
                break

            player = game.get_opponent(player)

    pygame.quit()



pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [20]:
model.eval()
play_against_model_pygame(model, game, device, play_as=1)


NameError: name 'play_against_model_pygame' is not defined